In [ ]:
import os
import json
import sqlite3
import pandas as pd
print(sqlite3.sqlite_version)

# Theory - basics

(Step 5)   SELECT   -- określanie kształtu wyniku, selekcja pionowa (kolumn)  
(Step 1)   FROM     -- określenie źródła (źródeł) i relacji między nimi  
(Step 2)   WHERE    -- filtracja rekordów  
(Step 3)   GROUP BY -- grupowanie rekordów   
(Step 4)   HAVING   -- filtrowanie grup   
(Step 6)   ORDER BY -- sortowanie wyniku  

# Basic SQL queries with python

In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("data/db/portal_mammals.sqlite")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM species;'):
    print(row)

In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("data/db/portal_mammals.sqlite")

cur = con.cursor()

# Return all results of query
cur.execute('SELECT plot_id FROM plots WHERE plot_type="Control"')
rows = cur.fetchall()
print(rows)

# Return first result of query
cur.execute('SELECT species FROM species WHERE taxa="Bird"')
row = cur.fetchone()
print(row)

# Be sure to close the connection
con.close()

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/db/portal_mammals.sqlite")
df = pd.read_sql_query("SELECT * from surveys", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

# More exercises with mock table

In [ ]:
main_df = pd.read_csv(r'./data/seeds_dataset_clean.txt', header=None, sep='\t')

In [ ]:
print(main_df.info())
print()
print(main_df.head(10))

Attribute Information (seeds dataset):

To construct the data, seven geometric parameters of wheat kernels were measured:
1. area A
2. perimeter P
3. compactness C = 4*pi*A/P^2
4. length of kernel
5. width of kernel
6. asymmetry coefficient
7. length of kernel groove

All of these parameters were real-valued continuous.

In [ ]:
# CREATE TABLE inside db
con = sqlite3.connect("data/db/seeds.sqlite")
cur = con.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS mock_table
(area, perimeter, compactness, length_of_kernel, width_of_kernel, asymmetry_coefficient, length_of_kernel_groove)''')

con.commit()

con.close()

In [ ]:
# INSERT some mock data
con = sqlite3.connect("data/db/seeds.sqlite")
cur = con.cursor()

cur.execute("INSERT INTO mock_table VALUES ('15.26',  '14.84',  '0.8710',  '5.763',  '3.312',  '2.221', '5.220')")

con.commit()

con.close()

In [ ]:
# See if it works (SELECT) ...
con = sqlite3.connect("data/db/seeds.sqlite")
cur = con.cursor()

# ... using iterator ...
print("iterator")
for row in cur.execute('SELECT * FROM mock_table'):
    print(row)
print()

# ... or using normal queries
print("query")
cur.execute("SELECT * FROM mock_table")
row = cur.fetchone()
print(row)
print(type(row))

con.close()

In [ ]:
# DROP mock table
con = sqlite3.connect("data/db/seeds.sqlite")
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS mock_table;")

try:
    cur.execute('SELECT * FROM mock_table')
except sqlite3.OperationalError:
    print("No table - deleted - good")
finally:
    con.close()

# Saving .csv file to seeds db

In [ ]:
con = sqlite3.connect("data/db/seeds.sqlite")
cur = con.cursor()

main_df.to_sql("seeds_params", con, if_exists="replace")

try:
    cur.execute('SELECT * FROM seeds_params')
    rows = cur.fetchmany(10)
    print(rows)
except sqlite3.OperationalError:
    print("No table - not good")
    
con.close()

## Working with actual .geojson file

In [ ]:
geojson_file = os.path.join('./data/geojsons/SKIERNIEWICE_SA2-WARSZAWA_SDN-OTS002.geojson')

with open (geojson_file) as f:
    geojson_data = json.load(f)

In [ ]:
print(geojson_data['type'])

In [ ]:
print(geojson_data['features'][0]['type'])

In [ ]:
print(geojson_data['features'][0]['properties']['fill-opacity'])

In [ ]:
print(geojson_data['features'][0]['properties']['stroke'])

In [ ]:
print(geojson_data['features'][0]['properties']['stroke-opacity'])

In [ ]:
print(geojson_data['features'][0]['geometry']['type'])

In [ ]:
print(geojson_data['features'][0]['geometry']['coordinates'][0:20])

Written above may be columns of db (??)  

Like this:

- type
- features
    - type
    - properties
        - fill-opacity
        - stroke
        - stroke-opacity
        - geometry
            - type
            - coordinates

In [ ]:
con = sqlite3.connect("data/db/geo.sqlite")
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS geo_table")
con.commit()

cur.execute('''CREATE TABLE IF NOT EXISTS geo_table (points)''')
con.commit()

cur.execute("INSERT INTO geo_table VALUES (?)", [data_for_db])
con.commit()

cur.execute("SELECT * FROM geo_table")
rows = cur.fetchall()
print(rows)

con.close()

Above doesn't work yet, because of wrong way of saving data to db (line 10, with INSERT)

Next attempt may be based on: 
https://stackoverflow.com/questions/16603621/how-to-store-json-object-in-sqlite-database

# Some more SQL with Khan Academy tutorial

# CREATE and INSERT - groceries

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS groceries (id INTEGER PRIMARY KEY, name TEXT, quantity INTEGER);")
con.commit()

con.close()

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()

# Run only once, because of "UNIQUE constraint failed: groceries.id"
cur.execute("INSERT INTO groceries VALUES (1, 'Bananas', 4);")
cur.execute("INSERT INTO groceries VALUES (2, 'Peanut Butter', 1);")
cur.execute("INSERT INTO groceries VALUES (3, 'Dark chocolate bars', 2);")
con.commit()

con.close()

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()

cur.execute("SELECT * FROM groceries;")
rows = cur.fetchall()

con.close()

print(rows)

### Querying - groceries

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS groceries;")
cur.execute('''
CREATE TABLE IF NOT EXISTS groceries (
    id INTEGER PRIMARY KEY, 
    name TEXT, 
    quantity INTEGER, 
    aisle INTEGER);
''')
con.commit()

# Run only once, because of "UNIQUE constraint failed: groceries.id"
cur.execute("INSERT INTO groceries VALUES (1, 'Bananas', 4, 7);")
cur.execute("INSERT INTO groceries VALUES (2, 'Peanut Butter', 1, 2);")
cur.execute("INSERT INTO groceries VALUES (3, 'Dark chocolate bars', 2, 2);")
cur.execute("INSERT INTO groceries VALUES (4, 'Ice Cream', 1, 12);")
cur.execute("INSERT INTO groceries VALUES (5, 'Peanut Butter', 6, 2);")
cur.execute("INSERT INTO groceries VALUES (6, 'Dark chocolate bars', 1, 4);")
con.commit()

con.close()

In [ ]:
def select(query: str, db_path: str="data/db/khan_acad.sqlite"):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    con.close()
    return rows

In [ ]:
print(select(query="SELECT * FROM groceries;", db_path="data/db/khan_acad.sqlite"))

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()
cur.execute("SELECT * FROM groceries;")
rows = cur.fetchall()
con.close()

print(rows)

In [ ]:
print(select(query="SELECT * FROM groceries ORDER BY aisle;"))

In [ ]:
print(select(query="SELECT * FROM groceries WHERE aisle > 5 ORDER BY aisle;"))

In [ ]:
print(f'total number of items: {select("SELECT SUM(quantity) FROM groceries;")}')

print(f'maximum number of single items class: {select("SELECT MAX(quantity) FROM groceries;")}')

print(f'total number of items by aisle: {select("SELECT aisle, SUM(quantity) FROM groceries GROUP BY aisle;")}')

### Bit more advanced querying - exercise_log

In [ ]:
con = sqlite3.connect("data/db/khan_acad.sqlite")
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS exercise_log;")
cur.execute('''
CREATE TABLE IF NOT EXISTS exercise_log (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    type TEXT, 
    minutes INTEGER,
    calories INTEGER,
    heart_rate INTEGER);
''')
con.commit()

# No problem with "UNIQUE constraint failed" now, because of AUTOINCREMENT
cur.execute("INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('biking', 30, 100, 110);")
cur.execute("INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('biking', 10, 30, 105);")
cur.execute("INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('dancing', 15, 200, 120);")
con.commit()

con.close()

print(query(" EXPLAIN QUERY PLAN SELECT * FROM exercise_log;"))

In [ ]:
print(query(" EXPLAIN QUERY PLAN SELECT * FROM exercise_log WHERE calories > 50 AND minutes < 30 ORDER BY calories;"))

In [ ]:
print(query("SELECT * FROM exercise_log WHERE calories > 50 OR minutes > 100;"))

# Apparently a better way of managing db connection:

In [ ]:
from contextlib import closing

In [ ]:
def query(sql, db_path="data/db/khan_acad.sqlite"):
    with closing(sqlite3.connect(db_path)) as con, con,  \
            closing(con.cursor()) as cur:
        cur.execute(sql)
        return cur.fetchall()

In [ ]:
# Adding more rows to exercise_log
query(sql="INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('tree climbing', 30, 70, 90);")
query(sql="INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('tree climbing', 25, 72, 80);")
query(sql="INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('rowing', 30, 70, 90);")
query(sql="INSERT INTO exercise_log(type, minutes, calories, heart_rate) VALUES ('hiking', 60, 80, 85);")

# Subqueries

### IN - drs_favorites

In [ ]:
# Outdoor activities
query("SELECT * FROM exercise_log WHERE type IN ('biking', 'hiking', 'tree climbing', 'rowing')")

In [ ]:
# Indoor activities
query("SELECT * FROM exercise_log WHERE type NOT IN ('biking', 'hiking', 'tree climbing', 'rowing')")

In [ ]:
query('''CREATE TABLE drs_favorites (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    reason TEXT
); ''')
query("INSERT INTO drs_favorites (type, reason) VALUES ('biking', 'Improvers endurance and flexibility');")
query("INSERT INTO drs_favorites (type, reason) VALUES ('hiking', 'Increases cardiovascular health');")

In [ ]:
query("SELECT * FROM drs_favorites;")

### AS, HAVING, SUM, AVG - exercise log

In [ ]:
query("SELECT type, SUM(calories) AS total_calories FROM exercise_log GROUP BY type;")

In [ ]:
query("SELECT type, SUM(calories) AS total_calories FROM exercise_log GROUP BY type HAVING total_calories > 100;")

In [ ]:
query("SELECT type FROM exercise_log GROUP BY type HAVING COUNT (*) >= 2;")

### IN - drs_favorites

In [ ]:
query("SELECT * FROM exercise_log WHERE type IN (SELECT type FROM drs_favorites);")

In [ ]:
query('''SELECT * FROM exercise_log WHERE type IN (
      SELECT type FROM drs_favorites WHERE reason LIKE ('%cardiovascular%'));''')

### CASE - exercise_log

In [ ]:
query("SELECT COUNT(*) FROM exercise_log WHERE heart_rate > 220-30;")

In [ ]:
query('''SELECT * FROM exercise_log WHERE 
      heart_rate > ROUND(0.50*190) 
      AND heart_rate  <= ROUND(0.90*190);''')

In [ ]:
query('''SELECT type, heart_rate,
    CASE 
        WHEN heart_rate > 190 THEN 'above max'
        WHEN heart_rate > 190*0.6 THEN 'above target'
        WHEN heart_rate > 190*0.5 THEN 'on spot'
        ELSE 'below target'
    END as 'hr_zone'
FROM exercise_log''')

In [ ]:
query('''SELECT COUNT(*),
    CASE 
        WHEN heart_rate > 190 THEN 'above max'
        WHEN heart_rate > 190*0.6 THEN 'above target'
        WHEN heart_rate > 190*0.5 THEN 'on spot'
        ELSE 'below target'
    END as 'hr_zone'
FROM exercise_log
GROUP BY hr_zone; ''')

Interesting (but not working) example of CASE from: 

https://www.khanacademy.org/computing/computer-programming/sql/more-advanced-sql-queries/pc/challenge-gradebook

In [ ]:
CREATE TABLE student_grades (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    number_grade INTEGER,
    fraction_completed REAL);
    
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Winston", 90, 0.805);
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Winnefer", 95, 0.901);
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Winsteen", 85, 0.906);
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Wincifer", 66, 0.7054);
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Winster", 76, 0.5013);
INSERT INTO student_grades (name, number_grade, fraction_completed)
    VALUES ("Winstonia", 82, 0.9045);

SELECT name, number_grade, ROUND(fraction_completed*100) AS percent_completed FROM student_grades;

SELECT COUNT (*),
    CASE 
        WHEN number_grade > 90 then "A"
        WHEN number_grade > 80 then "B"
        WHEN number_grade > 70 then "C"
        ELSE "F"
    END as "letter_grade"
FROM student_grades
GROUP BY letter_grade;

# Working with many tables

## JOIN

Examples based on Khan Academy course:

https://www.khanacademy.org/computing/computer-programming/sql/relational-queries-in-sql

### cross-join == cartesian product  
SELECT * FROM students, students_grades;  
  
### implicit inner-join  
SELECT * FROM students, students_grades  
    WHERE students.id = students_grades.student_id;  
      
### explicit inner-join  
SELECT * FROM students  
    JOIN students_grades  
    ON students.id = student_grades.student_id;  
  
### outer-join  
SELECT * FROM students  
    LEFT/RIGHT/FULL OUTER JOIN students_projects  
    ON students.id = students_projects.student_id;  
    
### self-join

### Think about it:
SELECT customers.name, customers.email, SUM(price)  
FROM orders  
LEFT OUTER JOIN customers  
ON customers.id = orders.customer_id  
GROUP BY customer_id;  

### UPDATE, DELETE

Good practice when using DELETE (?):

SELECT rows FROM table_name;  
and then change SELECT -> DELETE for safety:  
DELETE rows FROM table_name; 